In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

### Pobranie wektorów ze skryptów eeg_pacjenci i eeg_kontrola

Budowa tablic z wektorami:

Liczba wierszy = Liczba fragmentow
Liczba kolumn = Liczba elektrod * Liczba cech = 3 * 6 = 18

Przykładowy jeden wiersz (zachowana kolejnosc elektrod i cech):
- C3 aktywnosc
- C3 ruchliwosc
- C3 zlozonosc
- C3 srednia
- C3 wspolczynnik zmiennosci
- C3 liczba zerowych elementow
- Cz aktywnosc
- Cz ruchliwosc
- Cz zlozonosc
- Cz srednia
- Cz wspolczynnik zmiennosci
- Cz liczba zerowych elementow
- C4 aktywnosc
- C4 ruchliwosc
- C4 zlozonosc
- C4 srednia
- C4 wspolczynnik zmiennosci
- C4 liczba zerowych elementow

In [2]:
MCS_left_vectors = np.load('MCS-left-vectors.npy')
MCS_rest_vectors = np.load('MCS-rest-vectors.npy')
VS_left_vectors = np.load('VS-left-vectors.npy')
VS_rest_vectors = np.load('VS-rest-vectors.npy')

kontrola_left_vectors = np.load('kontrola-left-vectors.npy')
kontrola_rest_vectors = np.load('kontrola-rest-vectors.npy')

In [3]:
MCS_left_max_length = np.shape(MCS_left_vectors)[0]
VS_left_max_length = np.shape(VS_left_vectors)[0]
kontrola_left_max_length = np.shape(kontrola_left_vectors)[0]
print(MCS_left_max_length, VS_left_max_length, kontrola_left_max_length)
print(len(kontrola_left_vectors[0]))

print(np.all(MCS_left_vectors[0][50]) == 0)

14 16 18
48
True


## Polaczenie tych samych cech dla danej klasy w jeden wektor

In [4]:
kontrola_activity = []
kontrola_mobility = []
kontrola_complexity = []
kontrola_mean = []
kontrola_cov = []
kontrola_zero_cross = []

VS_activity = []
VS_mobility = []
VS_complexity = []
VS_mean = []
VS_cov = []
VS_zero_cross = []

MCS_activity = []
MCS_mobility = []
MCS_complexity = []
MCS_mean = []
MCS_cov = []
MCS_zero_cross = []


for patient in range(len(kontrola_left_vectors)):
    for period in range(len(kontrola_left_vectors[0])):
        for feature in range(18):
            if feature in [0, 6, 12]:
                kontrola_activity.append(kontrola_left_vectors[patient][period][feature])
            if feature in [1, 7, 13]:
                kontrola_mobility.append(kontrola_left_vectors[patient][period][feature])
            if feature in [2, 8, 14]:
                kontrola_complexity.append(kontrola_left_vectors[patient][period][feature])
            if feature in [3, 9, 15]:
                kontrola_mean.append(kontrola_left_vectors[patient][period][feature])
            if feature in [4, 10, 16]:
                kontrola_cov.append(kontrola_left_vectors[patient][period][feature])
            if feature in [5, 11, 17]:  
                kontrola_zero_cross.append(kontrola_left_vectors[patient][period][feature])
                
                
for patient in range(len(VS_left_vectors)):
    for period in range(len(VS_left_vectors[0])):
        for feature in range(18):
            if np.all(VS_left_vectors[patient][period]) == 0:
                break        
            if feature in [0, 6, 12]:
                VS_activity.append(VS_left_vectors[patient][period][feature])
            if feature in [1, 7, 13]:
                VS_mobility.append(VS_left_vectors[patient][period][feature])
            if feature in [2, 8, 14]:
                VS_complexity.append(VS_left_vectors[patient][period][feature])
            if feature in [3, 9, 15]:
                VS_mean.append(VS_left_vectors[patient][period][feature])
            if feature in [4, 10, 16]:
                VS_cov.append(VS_left_vectors[patient][period][feature])
            if feature in [5, 11, 17]:  
                VS_zero_cross.append(VS_left_vectors[patient][period][feature])


for patient in range(len(MCS_left_vectors)):
    for period in range(len(MCS_left_vectors[0])):
        for feature in range(18):
            if np.all(MCS_left_vectors[patient][period]) == 0:
                break        
            if feature in [0, 6, 12]:
                MCS_activity.append(MCS_left_vectors[patient][period][feature])              
            if feature in [1, 7, 13]:
                MCS_mobility.append(MCS_left_vectors[patient][period][feature])
            if feature in [2, 8, 14]:
                MCS_complexity.append(MCS_left_vectors[patient][period][feature])
            if feature in [3, 9, 15]:
                MCS_mean.append(MCS_left_vectors[patient][period][feature])
            if feature in [4, 10, 16]:
                MCS_cov.append(MCS_left_vectors[patient][period][feature])
            if feature in [5, 11, 17]:  
                MCS_zero_cross.append(MCS_left_vectors[patient][period][feature])
                
print(np.any(MCS_activity) == 0)
print(len(MCS_activity))

False
1476


## Parametr t dla aktywnosci

In [5]:
t_activity_kontrola_VS = np.abs(np.mean(kontrola_activity) - np.mean(VS_activity)) / np.sqrt((np.var(kontrola_activity) / len(kontrola_activity)) + (np.var(VS_activity) / len(VS_activity)))
t_activity_MCS_VS = np.abs(np.mean(MCS_activity) - np.mean(VS_activity)) / np.sqrt((np.var(MCS_activity) / len(MCS_activity)) + (np.var(VS_activity) / len(VS_activity)))
t_activity_kontrola_MCS = np.abs(np.mean(kontrola_activity) - np.mean(MCS_activity)) / np.sqrt((np.var(kontrola_activity) / len(kontrola_activity)) + (np.var(MCS_activity) / len(MCS_activity)))

t_activity = t_activity_kontrola_VS + t_activity_MCS_VS + t_activity_kontrola_MCS
print(t_activity)

19.064648067951886


## Parametr t dla ruchliwosci


In [6]:
t_mobility_kontrola_VS = np.abs(np.mean(kontrola_mobility) - np.mean(VS_mobility)) / np.sqrt((np.var(kontrola_mobility) / len(kontrola_mobility)) + (np.var(VS_mobility) / len(VS_mobility)))
t_mobility_MCS_VS = np.abs(np.mean(MCS_mobility) - np.mean(VS_mobility)) / np.sqrt((np.var(MCS_mobility) / len(MCS_mobility)) + (np.var(VS_mobility) / len(VS_mobility)))
t_mobility_kontrola_MCS = np.abs(np.mean(kontrola_mobility) - np.mean(MCS_mobility)) / np.sqrt((np.var(kontrola_mobility) / len(kontrola_mobility)) + (np.var(MCS_mobility) / len(MCS_mobility)))

t_mobility = t_mobility_kontrola_VS + t_mobility_MCS_VS + t_mobility_kontrola_MCS
print(t_mobility)

33.34561682958718


## Parametr t dla zlozonosci

In [7]:
t_complexity_kontrola_VS = np.abs(np.mean(kontrola_complexity) - np.mean(VS_complexity)) / np.sqrt((np.var(kontrola_complexity) / len(kontrola_complexity)) + (np.var(VS_complexity) / len(VS_complexity)))
t_complexity_MCS_VS = np.abs(np.mean(MCS_complexity) - np.mean(VS_complexity)) / np.sqrt((np.var(MCS_complexity) / len(MCS_complexity)) + (np.var(VS_complexity) / len(VS_complexity)))
t_complexity_kontrola_MCS = np.abs(np.mean(kontrola_complexity) - np.mean(MCS_complexity)) / np.sqrt((np.var(kontrola_complexity) / len(kontrola_complexity)) + (np.var(MCS_complexity) / len(MCS_complexity)))

t_complexity = t_complexity_kontrola_VS + t_complexity_MCS_VS + t_complexity_kontrola_MCS
print(t_complexity)

87.03712570904236


## Parametr t dla sredniej


In [8]:
t_mean_kontrola_VS = np.abs(np.mean(kontrola_mean) - np.mean(VS_mean)) / np.sqrt((np.var(kontrola_mean) / len(kontrola_mean)) + (np.var(VS_mean) / len(VS_mean)))
t_mean_MCS_VS = np.abs(np.mean(MCS_mean) - np.mean(VS_mean)) / np.sqrt((np.var(MCS_mean) / len(MCS_mean)) + (np.var(VS_mean) / len(VS_mean)))
t_mean_kontrola_MCS = np.abs(np.mean(kontrola_mean) - np.mean(MCS_mean)) / np.sqrt((np.var(kontrola_mean) / len(kontrola_mean)) + (np.var(MCS_mean) / len(MCS_mean)))

t_mean = t_mean_kontrola_VS + t_mean_MCS_VS + t_mean_kontrola_MCS
print(t_mean)

4.3111445494194855


## Parametr t dla wspolczynnika zmiennosci


In [9]:
t_cov_kontrola_VS = np.abs(np.mean(kontrola_cov) - np.mean(VS_cov)) / np.sqrt((np.var(kontrola_cov) / len(kontrola_cov)) + (np.var(VS_cov) / len(VS_cov)))
t_cov_MCS_VS = np.abs(np.mean(MCS_cov) - np.mean(VS_cov)) / np.sqrt((np.var(MCS_cov) / len(MCS_cov)) + (np.var(VS_cov) / len(VS_cov)))
t_cov_kontrola_MCS = np.abs(np.mean(kontrola_cov) - np.mean(MCS_cov)) / np.sqrt((np.var(kontrola_cov) / len(kontrola_cov)) + (np.var(MCS_cov) / len(MCS_cov)))

t_cov = t_cov_kontrola_VS + t_cov_MCS_VS + t_cov_kontrola_MCS
print(t_cov)

4.163880493723004


## Parametr t dla liczby zerowych elementow


In [10]:
t_zero_cross_kontrola_VS = np.abs(np.mean(kontrola_zero_cross) - np.mean(VS_zero_cross)) / np.sqrt((np.var(kontrola_zero_cross) / len(kontrola_zero_cross)) + (np.var(VS_zero_cross) / len(VS_zero_cross)))
t_zero_cross_MCS_VS = np.abs(np.mean(MCS_zero_cross) - np.mean(VS_zero_cross)) / np.sqrt((np.var(MCS_zero_cross) / len(MCS_zero_cross)) + (np.var(VS_zero_cross) / len(VS_zero_cross)))
t_zero_cross_kontrola_MCS = np.abs(np.mean(kontrola_zero_cross) - np.mean(MCS_zero_cross)) / np.sqrt((np.var(kontrola_zero_cross) / len(kontrola_zero_cross)) + (np.var(MCS_zero_cross) / len(MCS_zero_cross)))

t_zero_cross = t_zero_cross_kontrola_VS + t_zero_cross_MCS_VS + t_zero_cross_kontrola_MCS
print(t_zero_cross)

37.603972372397344


# Selekcja cech

In [11]:
selected_features = {"Aktywnosc": t_activity, 
                    "Ruchliwosc": t_mobility, 
                    "Zlozonosc": t_complexity, 
                    "Srednia": t_mean, 
                    "Wspolczynnik zmiennosci": t_cov, 
                    "Liczba zerowych elementow": t_zero_cross}

sorted_features = {key: value for key, value in reversed(sorted(selected_features.items(), key=lambda item: item[1]))}
for key, value in sorted_features.items():
    print(str(key) + ": %.3f" % value)

Zlozonosc: 87.037
Liczba zerowych elementow: 37.604
Ruchliwosc: 33.346
Aktywnosc: 19.065
Srednia: 4.311
Wspolczynnik zmiennosci: 4.164


# KLASYFIKACJA

### Przygotowanie wektorow


In [12]:
all_vectors_kontrola = []
all_vectors_MCS = []
all_vectors_VS = []

for patient in range(len(kontrola_left_vectors)):
    for period in range(len(kontrola_left_vectors[0])):
        all_vectors_kontrola.append(kontrola_left_vectors[patient][period])
        all_vectors_kontrola.append(kontrola_rest_vectors[patient][period])

for patient in range(len(VS_left_vectors)):
    for period in range(len(VS_left_vectors[0])):
        if np.all(VS_left_vectors[patient][period]) == 0:
            break        
        all_vectors_VS.append(VS_left_vectors[patient][period])
        all_vectors_VS.append(VS_rest_vectors[patient][period])


for patient in range(len(MCS_left_vectors)):
    for period in range(len(MCS_left_vectors[0])):
        if np.all(MCS_left_vectors[patient][period]) == 0:
            break   
        all_vectors_MCS.append(MCS_left_vectors[patient][period])
        all_vectors_MCS.append(MCS_rest_vectors[patient][period])

        
data = [all_vectors_kontrola, all_vectors_MCS, all_vectors_VS]
        
print(len(all_vectors_kontrola))
print(len(all_vectors_MCS))
print(len(all_vectors_VS))

1728
984
1368


In [13]:
# class_names = ['Healthy', 'MCS', 'VS']
class_names = ['MCS', 'VS']
#creating labelEncoder
le = preprocessing.LabelEncoder()
#converting string labels into numbers
class_names_encoded=le.fit_transform(class_names)

print(class_names)
print(class_names_encoded)
print(np.shape(class_names_encoded))

['MCS', 'VS']
[0 1]
(2,)


In [14]:
# features = []
# labels = []

# for i, dataset in enumerate(new_data):
#     for row in range(len(dataset)):
#         feature_vec = dataset[row]
#         features.append(feature_vec)
#         labels.append(class_names_encoded[i])


# # X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True)

# to_shuffle = list(zip(features, labels))
# random.shuffle(to_shuffle)

# features, labels = zip(*to_shuffle)

# print('Lenght of set: ' + str(len(features)))

# Klasyfikacja KNN

In [15]:
new_data = [all_vectors_MCS, all_vectors_VS]
class_names = ['MCS', 'VS']

le = preprocessing.LabelEncoder()
class_names_encoded=le.fit_transform(class_names)

features = []
labels = []

for i, dataset in enumerate(new_data):
    for row in range(len(dataset)):
        feature_vec = dataset[row]
        features.append(feature_vec)
        labels.append(class_names_encoded[i])

In [23]:
########## KNN ###########
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True)

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_test, y_pred)

print("Dokładność = %.3f" % accuracy_score(y_test, y_pred))
print("Precyzja = %.3f" % precision_score(y_test, y_pred, average='binary'))
print("Czułość = %.3f" % recall_score(y_test, y_pred, average='binary'))
print("F1 score = %.3f" % f1_score(y_test, y_pred, average='binary'))

Dokładność = 0.766
Precyzja = 0.789
Czułość = 0.807
F1 score = 0.798
